In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
pal_url = 'https://bulbapedia.bulbagarden.net/wiki/Paldea_Evolved_(TCG)'
pal_page = requests.get(pal_url)

In [3]:
pal_soup = BeautifulSoup(pal_page.text, 'html')

In [4]:
# Incidentally, there is another table with the same formatting.
pal_table = pal_soup.find_all('table', class_ = 'roundy', style = 'float:left; background:#98482a; border: 3px solid #651874')[1]

In [6]:
#Creating Table Headers by specifying the top row.
pal_headers = pal_table.find('tr', style = 'background:#faf5c9;')

In [7]:
#Strip the HTML tags for a List of column titles. Removing unnecessary column titles for this project.
pal_column_titles = [header.text.strip() for header in pal_headers.find_all('th')]
to_remove = ['Type', 'Mark', 'Promotion']
for title in to_remove:
    while title in pal_column_titles:
        pal_column_titles.remove(title)

In [9]:
#Create the dataframe with columns set.
pal_df = pd.DataFrame(columns = pal_column_titles)

In [11]:
#Incidentally, the type of the card is tagged as th, not td. It's unneeded for the purposes of this dataframe however.
pal_column_data = pal_table.find_all('td', style = 'background:#FFFFFF')

In [13]:
#Populate dataframe with column data. {'1': 'No.', '2': Mark, '3': 'Card name', '4': 'Rarity'}
column_number = 0
current_entry = []

for value in pal_column_data:
    row_to_update = len(pal_df)
    column_number += 1

    if column_number == 1:
        current_entry.append(value.text.strip())

    #Column 2, the Mark, is unnecessary so it's skipped.

    elif column_number == 3:
        #The ex (and other tags such as VStar) will not show up without additional code.
        card_name = value.text.strip()

        try:
            pokemon_tag = value.find_all('a')[1].get('title')
            card_name += ' ' + pokemon_tag
            current_entry.append(card_name)
        except:
            current_entry.append(card_name)

    elif column_number == 4:
        current_entry.append(value.find('a').get('title'))
        pal_df.loc[row_to_update] = current_entry

        #Reset for the next row.
        current_entry = []
        column_number = 0

In [15]:
pal_df.to_csv(r'C:/Users/zsere/Documents/Raw Data/Pokemon Set Card Info/Paldea Evolved Card Info.csv')